# 进展报告自动生成

基于项目文件（GitHubClient）调用大模型（LLM）自动生成项目进展报告。

### 调用 OpenAI GPT 大模型

相比 GitHub REST API ，OpenAI 提供的大模型相关 API 迭代速度快，且不够稳定。

**GPT-4 很难能够准确的生成 OpenAI Client 相关代码**。

因此，GitHubSentinel 项目中 LLM 相关调用代码由人类编写😁。


## Prompt 优化测试

基于 `GithubClient` 模块获取的 Repo 最新进展，先在 ChatGPT 中尝试获取可用的提示词（Prompt）方案。

-  **完整的ChatGPT 对话记录【GitHubSentinel 提示词优化】**：https://chatgpt.com/share/28524ea6-2bf3-4ebe-b7d9-9c1ba5f005d2
- 以下测试使用的 LangChain 项目文件为: `./daily_progress/langchain-ai_langchain/2024-08-18.md'`


### ChatGPT（GPT-4） 生成报告

**Langchain-AI/Langchain Daily Progress Report - 2024-08-18**

### 新增功能
1. **Langchain 模块添加**
   - 新增了Langchain Box套件及其文档加载器 (`langchain-box: add langchain box package and DocumentLoader` #25506)
   - 加入了新的社区提供者—Agentic RAG 示例 (`Community: Add Union provider - Agentic RAG example` #25509)
   - 引入了更多的异步测试标准 (`standard-tests[patch]: async variations of all tests` #25501)
   - 引入了对多种区块链的支持 (`community: add supported blockchains to Blockchain Document Loader` #25428)
   
2. **文档与API更新**
   - 更新了多个集成参考文档和Langchain版本的文档 (`docs: `integrations` reference update 9` #25511, `docs 0.3 release` #25459)
   - 增加了新的文档索引和数据加载方式的说明 (`[docs]: more indexing of document loaders` #25500)

### 主要改进
1. **测试与标准化**
   - 添加了更多嵌入标准测试 (`more embeddings standard tests` #25513)
   - 引入了JSON模式的标准测试 (`json mode standard test` #25497)
   - 新增了各种文档加载器的文档 (`[Doc] Add docs for `ZhipuAIEmbeddings`` #25467)

2. **框架和规则改进**
   - 对Langchain核心模块进行了Pydantic解析器修复 (`langchain-core: added pydantic parser fix for issue #24995` #25516)
   - 增加了B(bugbear) ruff规则以提高代码质量 (`core: Add B(bugbear) ruff rules` #25520)

3. **集成与兼容性**
   - 测试了Pydantic 2和Langchain 0.3的兼容性 (`openai[major] -- test with pydantic 2 and langchain 0.3` #25503)
   - 准备了向Pydantic 2迁移的根验证器升级 (`openai[patch]: Upgrade @root_validators in preparation for pydantic 2 migration` #25491)

### 修复问题
1. **错误修复**
   - 修正了文档中的错别字和错误消息 (`docs: Fix typo in openai llm integration notebook` #25492, `docs: fix Agent deprecation msg` #25464)
   - 解决了不同的搜索模式（向量与文本）产生不同结果的问题 (`Chroma search with vector and search with text get different result using the same embedding function` #25517)
   - 修复了使用AzureSearch vectorstore时的文档ID作为键的问题 (`community : [bugfix] Use document ids as keys in AzureSearch vectorstore` #25486)

2. **系统错误与异常处理**
   - 解决了在调用特定链时缺少输入键的错误 (`Raises ValueError: Missing some input keys: {'query'} everytime I invoke 'GraphCypherQAChain.from_llm' chain with query present as input keys` #25476)
   - 修正了未知类型 'ToolMessage' 的类型错误 (`TypeError: Got unknown type 'ToolMessage'.` #25490)

---

此简报详细总结了Langchain AI项目在2024年8月18日的最新进展，包括新增功能、主要改进和问题修复，确保团队成员了解最新的项目状态和即将到来的更新。


## 前置依赖 logger 模块

In [8]:
# src/logger.py
from loguru import logger
import sys

# Configure Loguru
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="DEBUG", format="{time} {level} {message}", colorize=True)
logger.add("logs/app.log", rotation="1 MB", level="DEBUG")

# Alias the logger for easier import
LOG = logger

# Make the logger available for import with the alias
__all__ = ["LOG"]

## LLM Class 

In [27]:
import os
import json
from openai import OpenAI  # 导入OpenAI库用于访问GPT模型
# from logger import LOG  # 导入日志模块

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取 DeepSeek API密钥

class LLM:
    def __init__(self, model="deepseek-chat"):
        self.model = model
        if self.model == "deepseek-chat":
            self.client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")
        else:
            # 创建一个OpenAI客户端实例
            self.client = OpenAI()
            # 从TXT文件加载提示信息
        with open("../../prompts/report_prompt.txt", "r", encoding='utf-8') as file:
            self.system_prompt = file.read()
        # 配置日志文件，当文件大小达到1MB时自动轮转，日志级别为DEBUG
        LOG.add("logs/llm_logs.log", rotation="1 MB", level="DEBUG")

    def generate_daily_report(self, markdown_content, dry_run=False):
        # 使用从TXT文件加载的提示信息
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": markdown_content},
        ]

        if dry_run:
            # 如果启用了dry_run模式，将不会调用模型，而是将提示信息保存到文件中
            LOG.info("Dry run mode enabled. Saving prompt to file.")
            with open("daily_progress/prompt.txt", "w+") as f:
                # 格式化JSON字符串的保存
                json.dump(messages, f, indent=4, ensure_ascii=False)
            LOG.debug("Prompt saved to daily_progress/prompt.txt")
            return "DRY RUN"

        # 日志记录开始生成报告
        LOG.info("Starting report generation using GPT model.")
        
        try:
            # # 调用 DeepSeek API 生成报告
            # if self.model == "deepseek-chat":
            #     response = self.client.chat.completions.create(
            #         model="deepseek-chat",
            #         messages=messages
            #     )
            # else:
            #     # 调用OpenAI GPT模型生成报告
            #     response = self.client.chat.completions.create(
            #         model="gpt-4o-mini",  # 指定使用的模型版本
            #         messages=messages
            #     )
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            LOG.debug("GPT response: {}", response)
            # 返回模型生成的内容
            return response.choices[0].message.content
        except Exception as e:
            # 如果在请求过程中出现异常，记录错误并抛出
            LOG.error("An error occurred while generating the report: {}", e)
            raise


## ReportGenerator Class

In [28]:
# src/report_generator.py

import os
from datetime import date, timedelta
# from logger import LOG  # 导入日志模块，用于记录日志信息

class ReportGenerator:
    def __init__(self, llm):
        self.llm = llm  # 初始化时接受一个LLM实例，用于后续生成报告

    def export_daily_progress(self, repo, updates):
        # 构建仓库的日志文件目录
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)  # 如果目录不存在则创建
        
        # 创建并写入日常进展的Markdown文件
        file_path = os.path.join(repo_dir, f'{date.today()}.md')
        with open(file_path, 'w') as file:
            file.write(f"# Daily Progress for {repo} ({date.today()})\n\n")
            file.write("\n## Issues\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        return file_path

    def export_progress_by_date_range(self, repo, updates, days):
        # 构建目录并写入特定日期范围的进展Markdown文件
        repo_dir = os.path.join('daily_progress', repo.replace("/", "_"))
        os.makedirs(repo_dir, exist_ok=True)

        today = date.today()
        since = today - timedelta(days=days)  # 计算起始日期
        
        date_str = f"{since}_to_{today}"  # 格式化日期范围字符串
        file_path = os.path.join(repo_dir, f'{date_str}.md')
        
        with open(file_path, 'w') as file:
            file.write(f"# Progress for {repo} ({since} to {today})\n\n")
            file.write("\n## Issues Closed in the Last {days} Days\n")
            for issue in updates['issues']:
                file.write(f"- {issue['title']} #{issue['number']}\n")
            file.write("\n## Pull Requests Merged in the Last {days} Days\n")
            for pr in updates['pull_requests']:
                file.write(f"- {pr['title']} #{pr['number']}\n")
        
        LOG.info(f"Exported time-range progress to {file_path}")  # 记录导出日志
        return file_path

    def generate_daily_report(self, markdown_file_path):
        # 读取Markdown文件并使用LLM生成日报
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content)  # 调用LLM生成报告

        report_file_path = os.path.splitext(markdown_file_path)[0] + "_report.md"
        with open(report_file_path, 'w+') as report_file:
            report_file.write(report)  # 写入生成的报告

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志

    def generate_report_by_date_range(self, markdown_file_path, days):
        # 生成特定日期范围的报告，流程与日报生成类似
        with open(markdown_file_path, 'r') as file:
            markdown_content = file.read()

        report = self.llm.generate_daily_report(markdown_content)

        report_file_path = os.path.splitext(markdown_file_path)[0] + f"_report.md"
        with open(report_file_path, 'w+') as report_file:
            report_file.write(report)

        LOG.info(f"Generated report saved to {report_file_path}")  # 记录生成报告日志


## 调用 GPT-3.5-Turbo API 生成报告

In [29]:
# 实例化 LLM，并使用默认的 GPT-3.5-Turbo 模型
llm = LLM()

In [30]:
llm.model

'deepseek-chat'

In [31]:
# 实例化 ReportGenerator
report_generator = ReportGenerator(llm)

In [32]:
# 生成 LangChain 项目最近一日报告
report_generator.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-10-14_to_2024-10-16.md")

2024-10-16T22:05:27.163114+0800 INFO Starting report generation using GPT model.
2024-10-16T22:05:49.426673+0800 DEBUG GPT response: ChatCompletion(id='af6ef3df-7156-40b5-9081-37316e82a022', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="# LangChain 项目进展\n\n## 时间周期：2024-10-14至2024-10-16\n\n## 新增功能\n- 添加了对CLOB数据类型的支持（#27330）\n- 添加了Google Books工具（#27302）\n- 添加了Databricks API构建（#27374）\n\n## 主要改进\n- 更新了IBM `ChatWatsonx`文档（#27358）\n- 改进了`parse_result`方法中的比较逻辑（#27244）\n- 将Makefile迁移到Compose V2语法（#27241）\n- 清理了openapi中的mypy语法警告（#27370）\n- 更新了`integrations google`文档（#27218）\n\n## 修复问题\n- 修复了vLLM集成中的SamplingParams过滤问题（#27367）\n- 修复了KeyError: 'client'问题（#27345, #27344）\n- 修复了Azure集成中的问题（#27336）\n- 修复了document_loaders/youtube.py中的Pydantic AttributeError（#27320）\n- 修复了DuckDuckGoSearchRun导致的AttributeError（#27184）\n- 修复了ChatGroq中的“cannot pickle 'classmethod' object error”（#27144）\n- 修复了`Text2vecEmbeddings`中的拼写错误（#27183）\n- 修复了`PromptTemplate`文档中

## 调用 GPT-4-Turbo API 生成报告

In [5]:
# 实例化 LLM，并使用指定的 GPT-4-Turbo 模型
gpt_4 = LLM()

In [6]:
# 实例化 ReportGenerator, 并使用指定的 GPT-4-Turbo 模型
rg_gpt_4 = ReportGenerator(gpt_4)

In [7]:
# 生成 LangChain 项目最近一日报告
rg_gpt_4.generate_daily_report(
    markdown_file_path="daily_progress/langchain-ai_langchain/2024-10-13_to_2024-10-16.md")

2024-10-16T00:58:15.932497+0800 INFO Starting report generation using model: gpt-4o-mini.
2024-10-16T00:58:28.299083+0800 DEBUG gpt-4o-mini response: ChatCompletion(id='chatcmpl-AIfA9DT4n86H1XMrCjSW1Vm4Hjew2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### 项目进展简报 (2024-10-13 至 2024-10-16)\n\n#### 1）新增功能\n- **Langchain Azure集成**：新增对Azure平台的支持，增强了与云服务的整合。\n- **bm25检索模型保存和加载方法**：增加了bm25检索模型的保存和加载功能，提升了用户体验。\n- **Anthropic提示缓存功能**：新增对Anthropic的提示缓存功能以优化性能。\n\n#### 2）主要改进\n- **`parse_result`方法的比较功能**：在`parse_result`方法中添加了比较功能，提升了数据解析的准确性。\n- **文档更新**：对`integrations`文档进行了多项更新，提升了使用指南的准确性；修复了`ChatWatsonx`文档中的错误，确保精确性。\n- **代码结构重构**：重新构建了Arxiv搜索逻辑，以提高代码的可维护性和性能。\n\n#### 3）修复问题\n- **修复KeyError异常**：解决了在未提供客户信息时出现的KeyError异常，增强了系统的鲁棒性。\n- **Pydantic属性错误修复**：修复了在`document_loaders/youtube.py`中出现的Pydantic属性错误。\n- **修复Typos**：对文档中的拼写错误进行了修正，确保文档内容准确。\n- **解决数据库连接问题**：修复了Neo4j数据库的连接问题，确保查询功能正常。\n\n### 总结\n此次更新主要集中在新增功能和重大改进方面，同时也解决了一系列问题，以优化项目的

### Homework: 与ChatGPT深度对话，尝试使用 System role 提升报告质量和稳定性